In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../DATA/books.csv')

In [3]:
def textual_representation(row):
  textual_representation = f"""Title: {row['title']}
  Author: {row['authors']}
  Description: {row['description']}
  Categories: {row['categories']}
  Publishing Year: {row['published_year']}
  Average Rating: {row['average_rating']}
  Number of Pages: {row['num_pages']}
"""
  
  return textual_representation

In [5]:
print(df.iloc[:5].apply(textual_representation, axis=1).values[0])

Title: Gilead
  Author: Marilynne Robinson
  Description: A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebrat

In [6]:
df['textual_representation'] = df.apply(textual_representation, axis=1)

In [13]:
import faiss
import requests
import numpy as np

In [14]:
dim = 4096

index = faiss.IndexFlatL2(dim)

X = np.zeros((len(df['textual_representation']), dim), dtype=np.float32)

In [19]:
for i, representation in enumerate(df['textual_representation']):
    response = requests.post('http://localhost:11500/api/embeddings', json={
        'model': 'llama2',
        'prompt': representation,
    })
    response.raise_for_status()

    embedding = response.json().get('embedding', [])
    if len(embedding) != dim:
        print(f"Unexpected embedding shape: {len(embedding)} for index {i}")

    X[i] = np.array(embedding)

index.add(X)

KeyboardInterrupt: 

In [ ]:
faiss.write_index(index, 'index')

In [ ]:
index = faiss.read_index('index')

In [ ]:
df[df.title.str.contains('Harry Potter')]

In [ ]:
favorite_book = df.iloc[4533]

In [ ]:
print(favorite_book['textual_representation'])

In [ ]:
"""Title: Python Bible 7 in 1
Authors: Florian Dedov
Description: Python Bible 7 in 1: Volumes 1-7
"""

In [ ]:
res = requests.post('http://localhost:11500/api/embeddings', json={
    'model': 'llama2',
    'prompt': favorite_book['textual_representation'],
})

In [ ]:
embedding = np.array([res.json()['embedding']], dtype=np.float32)

In [ ]:
D, I = index.search(embedding, 5)

In [ ]:
best_matches = np.array(df['textual_representation'][I.flatten()])

In [ ]:
for match in best_matches:
    print(match)
    print()